# Wavelet transformation using SRAD filtered images and Guided Filter

This method follows the implementation of _Choi and Jeong, 2019, Speckle Noise Reduction Technique for SAR Images Using Statistical Characteristics of Speckle Noise and Discrete Wavelet Transform, Remote Sensing._ (https://www.mdpi.com/2072-4292/11/10/1184) Filter implementation is much faster in MATLAB or C++. 

The architecture as proposed: 

![title](https://www.mdpi.com/remotesensing/remotesensing-11-01184/article_deploy/html/images/remotesensing-11-01184-g001-550.jpg)



Disclaimer: Original paper does not provide DWT transformation, hence an arbitrary method is being used by author. The selected method ('HAAR') has been chosen as representative for the author's application. To be replaced as necessary. Moreover, the selected method does not provide filtering parameters, which are strongly dependent on the data, hence parameters might vary and are not represented as default. Please try your own parameters beforhand. Furthermore, this implementation uses the guided filter and not the improved guided filter method proposed by Choi and Jeong.

Using implementation from PyWavelets: https://pywavelets.readthedocs.io/en/latest/
_Gregory R. Lee, Ralf Gommers, Filip Wasilewski, Kai Wohlfahrt, Aaron O’Leary (2019). PyWavelets: A Python package for wavelet analysis. Journal of Open Source Software, 4(36), 1237, https://doi.org/10.21105/joss.01237_

Using implementation after _Yu and Acton, 2002, Speckle Reducting Anisotropic Diffusion, IEEE Transactions on Image Processing, vol 11, no. 11_. 

Using implementation by PfChai (https://github.com/pfchai/GuidedFilter) using original paper implementation and code in MATLAB by Kaiming He (http://kaiminghe.com/eccv10/, http://kaiminghe.com/publications/eccv10guidedfilter.pdf). The Python implementation of the code has been reviewed by author for correspondance against original MATLAB code and paper mathematical background.

Author: Cristina Vrinceanu, University of Nottingham, Nottingham Geospatial Institute, 2020

In [ ]:
import pywt  # imports Pywavelets library. Please install PyWavelets first as the implementation is dependend on it and read documentation.
from osgeo import gdal
import numpy as np
import matplotlib.pyplot as plt
import math
import os, glob
from guidedfilter import guidedfilter  #imports guided filter implementation from PfChai for the guided filter. Please install first.

In [ ]:
def dwt_filter(input_file, output_file):
    print('Begin filtering file ' + input_file)
    
    # inputing the file
    sar_image=gdal.Open(input_file)
    sar_band=sar_image.GetRasterBand(1)
    sar_band.GetMetadata()
    img_array=sar_band.ReadAsArray()
    [cols, rows]=img_array.shape
    
    #srad filtering. Parameters to be adjusted.
    
    def Gradient(img):
        """
        Gradients of one image with symmetric boundary conditons

        Parameters
        -------
        img ； ndarray

        Returns
        ------
        grx : ndarry
            one-order froward  difference in the direction of column(axis = 1)
        gry : ndarry
            one-order froward  difference in the direction of row   (axis = 0)
        glx : ndarry
            one-order backward difference in the direction of column(axis = 1)
        gly : ndarry
            one-order backward difference in the direction of row   (axis = 0)
        grc : ndarry
            self-defined difference function    
        """
        #img(i,j-1)
        img_right = np.roll(img,1,axis = 1)
        img_right[:,0] = img[:,0]
        #img(i,j+1)
        img_left  = np.roll(img,-1,axis = 1)
        img_left[:,-1] = img[:,-1]
        #img(i+1,j)
        img_up = np.roll(img,-1,axis = 0)
        img_up[-1] = img[-1]
        #img(i-1,j)
        img_down = np.roll(img,1,axis = 0)
        img_down[0] = img[0]

        #img(i,j+1) - img(i,j)
        grx = img_left - img 
        #img(i+1,j) - img(i,j)
        gry = img_up - img
        #img(i,j)  - img(i,j-1)
        glx = img - img_right 
        #img(i,j)   - img(i-1,j)
        gly = img - img_down   
        #img(i,j+1) + img(i+1,j)+ img(i,j-1) +img(i-1,j)  - 4*I(i,j)
        grc = grx+gry-glx-gly  
        return grx,gry,glx,gly,grc

    def qq(img):
        """
        Instantaneous coefficient of variation: q(x,y,t)

        Parameters
        ------
        img: ndarray

        Returns
        ------
        q : ndarray
            The formula is as follows:
            q(x, y ; t)=\sqrt{\frac{(1 / 2)(|\nabla I| / I)^{2}
            -\left(1 / 4^{2}\right)\left(\nabla^{2} I / I\right)^{2}}
            {\left[1+(1 / 4)\left(\nabla^{2} I / I\right)\right]^{2}}}
        """
        grx,gry,glx,gly,grc = Gradient(img)
        q_1 = (grx**2+gry**2+glx**2+gly**2)**0.5/(img+1e-06)
        q_2 = grc /(img+1e-06)
        q   = ((1/2*q_1**2 - 1/16*q_2**2) / ((1+1/4*q_2)**2)+1e-06)**0.5

        return q

    def srad(img,k = 30,m = 0.5,q_0 = 1,rho = 1,delta_t = 0.05,Iterations = 150):   #adjust parameters as needed. See Yu and Acton, 2002
        """
        speckle reducing anistropic diffusion

        Parameter
        ------
        img: ndarray

        k:  number
            attenuation coefficient
        m； number
            control rate of homogeneous area
        q_0: number
            the threshold of intial speckle noise
        rho: number
        delta_t:number
            timespace
        Iteration: number
            the number of iterations

        Returns
        img: ndarray 
            the image after being filtered by srad
        """
        for i in range(0,Iterations):
            grx,gry,glx,gly,grc = Gradient(img)

            # compute the diffusion coefficient
            q  = qq(img)
            q_t = q_0*math.exp(-rho*i*delta_t)
            cq = np.pi/2 - np.arctan(k*(q**2 - m*q_t**2))

            # cq(i+1,j)
            cq_up = np.roll(cq,-1,axis = 0)
            cq_up[-1] = cq[-1]
            # cq(i,j+1)
            cq_left = np.roll(cq,-1,axis = 1)
            cq_left[:,-1] = cq[:,-1]

            Div = cq_up*gry - cq*gly + cq_left*grx-cq*glx
            img = img + 1/4*delta_t*Div
        return img
    
    filtered_srad=srad(img_array)
    
    # transformation to logarithmic scale for converting from multiplicative to additive noise
    
    srad_log=np.log(filtered_srad)
    
    # discrete wavelet decomposition (uses HAAR for this decomposition but can be replaced as needed) at Level 2 decomposition cycle using Pywavelts
    
    coeffs = pywt.wavedec2(srad_log, 'haar', mode='periodization', level=2)
    
    #parameters for visualization of decomposition coefficients if need
    cA=coeffs[0]                   # approximation coefficient LL2 at level 2
    (cH1, cV1, cD1)= coeffs[-1]    # level 1 Horizontal, Vertical, Diagonal coefficients (LH1, HL1, HH1)
    (cH2, cV2, cD2)= coeffs[-2]    # level 2 Horizontal, Vertical, Diagonal coefficients (LH2, HL2, HH2)
    
    #soft thresholding of the Horizontal and Vertical coefficients (LH1, HL1 & LH2, HL2) using PyWavelets. Parameters to be adjusted.
    
    th_LH1=pywt.threshold(coeffs[-1][0],value=0.02, mode='soft')
    th_HL1=pywt.threshold(coeffs[-1][1],value=0.02, mode='soft')
    th_LH2=pywt.threshold(coeffs[-2][0],value=0.06, mode='soft')
    th_HL2=pywt.threshold(coeffs[-2][1],value=0.06, mode='soft')
    
    #applying the guided filter to the Diagonal coefficients (L1 & L2) and the LL2 Approximation. Parameters r and eps to be adjusted.
    
    flt_LL2=guidedfilter(coeffs[0],coeffs[0], r=5, eps=0.4 )
    flt_HH1=guidedfilter(coeffs[-1][2],coeffs[-1][2], r=5, eps=0.4 )
    flt_HH2=guidedfilter(coeffs[-2][2],coeffs[-2][2], r=5, eps=0.4)
    
    #reconstruct the Approximation coefficient for Level 2. Use the same parameters as in decomposition.
    reconstructed_LL2 = pywt.waverec2([flt_LL2,(th_LH2,th_HL2,flt_HH2)] , 'haar', mode='periodization')
    
    #reconstruct the Approximation coefficient for Level 1. Use the same parameters as in decomposition.
    reconstructed_LL1 = pywt.waverec2([reconstructed_LL2,(th_LH1,th_HL1,flt_HH1)] , 'haar', mode='periodization')
    
    #transformation to exponential scale for converting back to the original values after doing logarithmic transformation
    filtered_dwt=np.exp(reconstructed_LL1)
    
    #reshaping the image to get rid of the artificial padding added during DWT. Make sure to check how the chosen type of padding behaves beforehand.
    
    [cols1, rows1]= filtered_dwt.shape
        
    if rows==rows1:
        filtered_r=filtered_dwt
    else:
        filtered_r=filtered_dwt[:,:-1]
        
    [cols2, rows2]=filtered_r.shape     
    
    print(img_array.shape, filtered_dwt.shape, rows, cols, rows1, cols1, rows2, cols2)
    
    if cols==cols2:
        filtered_image=filtered_r
    else:
        filtered_image=filtered_r[:-1,:]
        
    # writing output file
    driver = gdal.GetDriverByName("GTiff")
    output = driver.Create(output_file, rows, cols, 1, gdal.GDT_Float32)
    proj = output.SetGeoTransform(sar_image.GetGeoTransform())  
    output.SetProjection(sar_image.GetProjection())
    output.SetGCPs(sar_image.GetGCPs(), "4326")  #use appropriate Coordinate System
    geoband = output.GetRasterBand(1)
    geoband.WriteArray(filtered_image)
    output.GetRasterBand(1).SetNoDataValue(-9999)
    output.FlushCache()
    output = None
    band=None
    print('Created file ' + output_file)
    
    print('End filtering file ' + input_file)

In [ ]:
for input_file in glob.glob('/home/cristina/Seeps/Workflow/Filters/ROI/*/8bit/*tif'):
    output_file = input_file.split(os.sep)
    output_file[-1] = output_file[-1][:-4] + '_DWT.tif'
    output_file = output_file[:-2] + ['Filters']+['DWT'] + output_file[-2:]
    output_file = os.sep.join(output_file)
    dwt_filter(input_file, output_file)
    
print("Filtering has been concluded!")